# Lab 1 
## EE120 Spring 2021
## Functional MRI analysis
Jingjia Chen, Naomi Sagan, Leyla Kabuli

Feb 8, 2020

# Table of Contents
* [Q0: What is fMRI](#q0)
   
* [Q1: Correlation](#q1)
    * [Q1.1: Implement cross correlation](#q11)
    * [Q1.2: test your code (1)](#q12)
    * [Q1.3: test your code (2)](#q13)
* [Q2: If brain is LTI](#q2)
    * [Q2.1: cross correlation, correlation coefficient (for test signal)](#q21)
    * [Q2.2 cross correlation analysis (for fMRI signal)](#q22)

<a id='q0'></a>
## 0. What is fMRI?
In this Lab, we will implement and perform correlation-based analysis on functional MRI.

Functional MRI (fMRI) is a non-invasive method for detecting real-time and regional brain activities. The fluctuation of the blood oxygen level reflects the activation of neurons within the region of blood supply. A measure of using cross correlations between sensory input and brain's heamodynamic response is commonly used for fMRI time series analysis<sup><cite>[1]</cite></sup>.



fMRI has widely been applied in many studies in cognitive and behaviour neuroscience, from monitoring neurodegenerative diseases progression <sup><cite>[2]</cite></sup>, understanding psychological pathology <sup><cite>[2]</cite></sup>, to decoding brain function <sup><cite>[3]</cite></sup>, etc.



First, let's load the data we will be working with.

In [ ]:
!pip3 install nibabel

In [ ]:
import nibabel as nib
import numpy as np
import scipy.signal as spsig
import scipy.stats as spstat
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
brain_img = nib.load('fMRI_ref_image.nii')
fMRI = nib.load('fMRI_slice.nii')
brain_img = brain_img.get_fdata()
fMRI = fMRI.get_fdata()

In [ ]:
brain_img.shape

In [ ]:
fMRI.shape

Now we have 2 matrices of brain images that we will be using for this lab. The first one, "brain_img" is one slice of brain image, that we will be using as a reference image for display purposes. The second one, "fMRI" is the time series images of the same slice being scanned and saved every 3 seconds. There are in total 120 time points. So you will know the total acquisition time is $3 \text{ sec} *120 = 360 \text{ seconds} = 6 \text{ min}$.

Let's take a look at the slice of brain we are working with.

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.imshow(brain_img,cmap='gray')
plt.colorbar()
plt.axis('off');

<a id='q1'></a>
## 1. Correlation

<a id='q11'></a>
### 1.1 Implement cross correlation
Suppose you are given 2 discrete time signal $x[n]$ and $y[n]$ of arbitraty length. Denote the cross correlation of $x[n]$ and $y[n]$ as $$R_{x,y}[n] = x[n]\circ y[n] = \sum_k x[k]y[k + n].$$

Based on this, we will implement a function to calculate the cross correlation between two signals.

To do so, we will have to give some thought to the dimensions of the input and output signals.

#### For the implementation of this, there are a few things you should keep in mind:
1. What will the length of the output signal be in terms of $N = \text{len}(x[n])$ and $M = \text{len}(y[n])$? If you aren't sure, try correlating a few length-2 or 3 signals manually and see what the length of the output is (where is it nonzero?).
2. Each index $n$ of the correlation is the element-wise multiplication of $x[k]$ with a version of $y[k]$ shifted $n$ to the left. In order to implement this easily using Numpy, we must <b> zero-pad </b> our signals to target output signal length, i.e. concatenate $x[k]$ and $y[k]$ with arrays of zeros using <a href="https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html"> np.concarenate</a>.
  * Think of the first index of the cross correlation function, which corresponds to the smallest $n$ such that the correlation is nonzero. For how many indices to the left or right of $x[k]$ does $y[k + n]$ extend? Zero-pad $x[k]$ by that amount. For instance, if $y[k + n]$ extends 1 index past the right of $x[k]$, you would add one $0$ to the right of $x$.
  * Follow the same process for the last index of the convolution.
  * Finally, how can you zero-pad $y$ so that it is the same length of $x$? Does it make sense to pad $y$ with zeros to the left or the right? The padding for $y$ will need to change as you will need to "slide" $y[k]$ along the axis of $k$ in order to calculate for different $n$.

In [ ]:
def my_xcorr(xx,yy):
    ## your code here
    # the total length of the resultant cross correlation is length(xx)+length(yy)-1        
    # Padding zeros according to the process outlined in the instructions
    # Define a Numpy array for the output.
    # Shift yy_pad to the left to achieve y[k+n]
    return corr

<a id='q12'></a>
### 1.2 Test your code (1)
Now we need to test if your code correctly compute the cross corrlation of two signal. For the following 2 signals provided in form of 1D array. Treat them as input of your function, and compare your result with the result from correlation function in Numpy module.

<strong> Note: Numpy defines correlation differently than we do in this class, so you must <em>switch the order of two inputs</em> when doing correlations with Numpy.</strong>

In [ ]:
# signals to compute the cross-correlation
signal1 = np.array([0, 1, 2, 3, 3, 2, 1, 0],dtype='double')
signal2 = np.array([1, 2, 3, 1, 1],dtype='double')

In [ ]:
print(my_xcorr(signal1,signal2))

In [ ]:
# Numpy correlation. Note that signal2 is the first input instead of signal1.
print(np.correlate(signal2, signal1, 'full'))

Does your result match the result from Numpy?

In [ ]:
np.equal(np.correlate(signal2, signal1, 'full'),my_xcorr(signal1,signal2)).all()

<a id='q13'></a>
### 1.3 Test your code (2)

In homework 3, problem 6, you were given 2 signals $$x(t) = \begin{cases} 0 & t<0 \text{ and } t>3 \\2& 0<t<3\end{cases}$$ and $$g(t) = e^{-t/2}u(t)$$

<b> Now, use the `my_xcorr` function you wrote to compute the correlations from problem 6.2 ($R_{x, g}(t)$) and 6.3 ($R_{g, x}(t)$). </b>


Note that computers cannot deal with a continuous range of points, so you will need to discretize the time variable $t$ with very small intervals to approach the continuous time signal appearance. To do so, look into the <a href="https://numpy.org/doc/stable/reference/generated/numpy.linspace.html"> np.linspace </a> function to produce a range of time values.

Plot $R_{x,g}(t)$ and $R_{g,x}(t)$ for $t\in (-10,10)$.

<b> Hints: </b>
* You might notice that your correlation plots are the same shape as what you got in the homework, but scaled vertically by a large amount. This is because you are estimating the integral equation for correlation as a sum over many very closely-spaced points. This is close to the Riemann sum approximation of an integral ($\int y\,dt \approx  \sum y\delta t$) but missing one thing. How can you make your method match the Riemann sum method?
* The output will cover a different range of times than the inputs. Keep this in mind.

In [ ]:
## your code here
## define two signals x and g
## calculate cross correlation 

# rxg = my_xcorr(x,g)*dt
# rgx = my_xcorr(g,x)*dt
    # rememeber to multiply the result with dt, the sampled time interval
    # as we are using discrete array to approach a continuous signal
    # the cross correlation formular for continuous time has a 'dt' inside integral
    # for discrete time case, dt is automatically treated as 1

## plot your cross correlation rxg and rgx

<a id='q2'></a>
## 2.  If the brain is LTI

Now you have implemented a very powerful tool, cross correlation, which is widely applied in functional MRI analysis research. Let's assume the brain is a LTI system. Note that the brain is not completely LTI, as you can argue considering that your reaction to a horror movie is different if you watch it today or if you watched it back when you were 7 years old. However, for simple input (for example, eating a sour blueberry), within short period of time (with in minutes), this assumption is valid.

For a typical LTI system, if the input include a certain pattern, the output will be highly correlated with the pattern found in the input. This means that, if we perform a cross-correlation of the output and the input, one or more indices of the correlation will be reasonably large. In other words, there will be one or more significant peaks.


<a id='q21'></a>
### 2.1 Cross correlation and correlation coefficients (for test signals)
It is common practice in time series analysis to normalize the cross-correlation function to get a time-dependent Pearson correlation coefficient. In our case, in order to make use of the cross correlation function we implemented, instead, we will send **zero centered (zero-mean) and normalized** signals to the function. Then the maximum value of cross correlation is defined as the *Pearson correlation coefficient*. 

**The closer the Pearson coeffecient is to 1, the more highly-correlated two signals are.**

Consider a system the produces output $y[n] = 5x[n+1]$ for input $x[n]$. For $x[n] = 2\cos[n/5]$ as the input to this LTI system, the corresponding output is $y[n] = 10\cos[n/5 + 1/5]$. Consider the range of $n\in [-30,30]$.

First, we need to shift $x[n]$ and $y[n]$ so that they are zero-centered, aka zero mean.

In [ ]:
## your code here
    # define x and y
    # center x and y

Secondly, normalize the centered versions of x and y such that their L2-norm is 1. Hint: use <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html">np.linalg.norm</a>.

In [ ]:
## your code here
    # normalize x and y

If your normalization procedure (step 1. zero mean, step 2. scale by norm) is done properly, the variance of each signal should be 1. Test with your my_xcorr function. The variance of a signal is equivalent to the maximum value of normalized autocorrealtion (cross correlation of a signal to itself).

In [ ]:
# test if the variance of normalized x is 1
# my_xcorr(normalized x, normalized x).max()

In [ ]:
# test if the variance of normalized y is 1
# my_xcorr(normalized y, normalized y).max()

Now, with zero centered and normalized signals, let's calculate the Pearson correlation coefficient of them. Complare your result with Scipy built-in function scipy.stats.pearsonr. 

We have *imported scipy.stats as spstat* in the very beginning, so just use *spstat.pearsonr*, and the first return is the Pearson correlation coefficient. (*spstat.pearsonr(x,y)[0]*)

You may see a slight difference (<0.001) between your version and the Scipy version. This is the result of numerical error.

In [ ]:
## correlation coefficient by my_xcorr
# my_xcorr(normalized x, normalized y).max()

In [ ]:
## correlation coefficient by scipy built-in function
# spstat.pearsonr(normalized x, normalized y)[0]

You will see that the input and output are highly correlated, with a Pearson coefficient close to 1. 

Now assume that at some moment, the system breaks down, and starts to yield random numbers as output. Use <a href="https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html">numpy.random.rand</a> to simulate the situation, what is the correlation coefficient now? (*remember to zero-center and normalize your random number output!*)

In [ ]:
## make a random output
## my_xcorr(normalized x, normalized random output).max()

The output is hardly correlated with the input when the system is broken and outputing random noise.

<a id='q22'></a>
### 2.2 Cross correlation analysis (for fMRI signals)
We have loaded an three-dimension array of data named "fMRI". The three dimensions are x, y and time. The data was acquired while the subject performed an auditory guided finger tapping experiment. A blocked design scheme was used: 15 seconds rest, 15 seconds finger tapping, 15 seconds rest, 15 seconds, finger tapping,... repeat until 6 min is reached. Every 3 seconds, we acquire the brain image data once. Therefore, in total, 6 min, we have 120 brain slice images, and 30 seconds is the period of the task (input).

In [ ]:
import numpy.matlib

In [ ]:
reference = np.squeeze(np.matlib.repmat(np.array([0,0,0,0,0,1,1,1,1,1]),1,12))
fig = plt.figure(figsize=(15, 3))
plt.title("Input signal")
plt.stem(np.arange(1,121),reference)
plt.yticks([0, 1],['Off', 'On'])
plt.show

Treat this test guide as the input to the brain. The time series signal at each pixel (x, y) of the brain image will then be the output of that small chunk of brain. From intuition, we know that, some region of brain will be more associated with limb (finger) movement than other region. Since correlation coefficient is a metric indicating how well two signals are correlated, if we correlate time series of each pixel in brain to the input reference signal, we will see which region is more related to the singer tapping task.

**Your job:** for each pixel, find the correlation coefficient using my_xcorr that you implemented in Q1. Your final result will be presented as a correlation image of the brain. (Remember to **center and normalize** your time series signals before feeding them into `my_xcorr`.)

If you want to go fancy, you can overlay your correlation coefficient result on top of the brain slice image *brain_img* by using <a href="https://numpy.org/doc/stable/reference/generated/numpy.ma.masked_array.html">np.ma.masked_array</a>.

(The images in the *fMRI* 3D array have already gone through regular fMRI preprocessing including motion correction, registration, etc.)

In [ ]:
## your code here

If you are familiar with brain anatomy, you will realize that this highly activated region locates at [motor cortex](https://en.wikipedia.org/wiki/Motor_cortex)<sup><cite>[5]</cite></sup>.

### Reference

[1]: Friston, K. J., et al. (1994). Human brain mapping, 1(2), 153-171.

[2]: Sen, S., et al. (2010). Neuroscience, 166(2), 712-719.

[3]: Cortese, S., et al. American Journal of Psychiatry 169.10 (2012): 1038-1055.

[4]: Gaillard, W. D., et al. Neurology 63.8 (2004): 1403-1408.

[5]:  Wikipedia, Motor cortex